In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
import ants

In [ ]:
rcParams_tmp = plt.rcParams["figure.figsize"]
print(plt.rcParams["figure.figsize"])
plt.rcParams["figure.figsize"] = [i * 3 for i in rcParams_tmp]
print(plt.rcParams["figure.figsize"])

In [ ]:
def get_ants_image(path, subject, run, skip_frame=0):
    for file_path in path.glob(f'{subject}/*/*/*task-fastEPI_run-{run}*.nii.gz'):
        image = ants.image_read(str(file_path))
        return image[..., 0, skip_frame:], file_path

In [ ]:
def get_fixed_image(datas):
    fixed = np.mean(datas, axis=2)
    fixed_image = ants.from_numpy(fixed)
    return fixed_image

In [ ]:
def get_transform_parameters(regis_file):
    # np.array((0.9,0.1,0.2,1.1,10,11)).reshape(2,3)
    # array([[ 0.9,  0.1,  0.2],
           # [ 1.1, 10. , 11. ]])
    y_s, y_r, x_r, x_s, y_t, x_t = ants.read_transform(regis_file).parameters
    return x_r, y_r, x_t, y_t

In [ ]:
from sklearn.metrics import mean_squared_error
def motion_detection(moving, fixed, frame=0):
    moving_image = ants.from_numpy(moving)
    fixed_image = ants.from_numpy(fixed)
    registration = ants.registration(fixed=fixed_image, moving=moving_image, aff_metric='meansquares', type_of_transform='Rigid')  # Rigid  Translation
    moved_image = registration['warpedmovout']

    moving = moving
    moved = moved_image[:]
    fixed = fixed
    mse = mean_squared_error(moving - fixed, moved - fixed)
    
    fwdt_file = registration['fwdtransforms'][0]
    y_s, y_r, x_r, x_s, y_t, x_t = ants.read_transform(fwdt_file).parameters
    os.system(f'rm {fwdt_file}')
    return frame, mse, y_s, y_r, x_r, x_s, y_t, x_t, moved_image[:]

In [ ]:
import time
from multiprocessing import Pool

subject = 'sub-07142303'
run = '05'

data_path = Path('/run/user/1000/gvfs/sftp:host=30.30.30.17,user=youjia/mnt/ngshare2/TRIGGER/BIDS')
output_path = Path('/run/user/1000/gvfs/sftp:host=30.30.30.17,user=youjia/mnt/ngshare2/TRIGGER/Preprocessing')
fixed_image = get_fixed_image(get_ants_image(data_path, subject, run, skip_frame=1000)[0])

# for run in ['05', '06', '07', '08', '09']:
for run in ['08']:
    moving, moving_file_path = get_ants_image(data_path, subject, run, skip_frame=1000)
    moved_file = output_path / subject / moving_file_path.name.replace('.nii.gz', '_mc.nii.gz')
    print(moving_file_path)
    print(moved_file)
    if not moved_file.parent.exists():
        moved_file.parent.mkdir(parents=True, exist_ok=True)
    if moved_file.exists():
        print(moved_file, ': exists')
        continue
    
    cmd_list = list() 
    for i in range(0, 10000):
        cmd_list.append([moving[..., i], fixed_image[:], i])
    time_s = time.time()
    pool = Pool(20)
    results = pool.starmap(motion_detection, cmd_list)
    del cmd_list
    pool.close()
    pool.join()
    print(time.time() - time_s)
    
    results = np.array(results, dtype=object)
    datas = {
    'frame': results[:, 0].astype(float),
    'mse': results[:, 1].astype(float),
    'x_r': results[:, 4].astype(float),
    'y_r': results[:, 3].astype(float),
    'x_t': results[:, 7].astype(float),
    'y_t': results[:, 6].astype(float),
    'x_s': results[:, 5].astype(float),
    'y_s': results[:, 2].astype(float),
    }
    datas['abs_dis'] = np.sqrt(np.power(datas['x_t'], 2), np.power(datas['y_t'], 2))
    datas['rel_dis'] = datas['abs_dis'].copy()
    datas['rel_dis'][1:] = np.abs(datas['abs_dis'][1:] - datas['abs_dis'][:-1])
    datas['rel_dis'][0] = 0
    motion_df = pd.DataFrame(datas, index=datas['frame'])
    motion_df[['x_t', 'y_t']].plot()
    plt.show()
    motion_df[['x_r', 'y_r']].plot()
    plt.show()
    motion_df[['mse']].plot()
    plt.show()
    motion_df[['abs_dis', 'rel_dis']].plot()
    plt.show()

    moved_mc_csv_file = output_path / subject / moving_file_path.name.replace('.nii.gz', '_mc.csv')
    motion_df.to_csv(moved_mc_csv_file, index=False)
    
    moving_nib_img = nib.load(str(moving_file_path))
    moved = np.array(list(results[:, 8]))
    moved = moved[..., np.newaxis]
    moved = np.transpose(moved, [1, 2, 3, 0])
    moved = np.concatenate((moving_nib_img.dataobj[..., :1000], moved), axis=3)
    
    moved_nib_img = nib.Nifti1Image(moved, moving_nib_img.affine, moving_nib_img.header)
    
    nib.save(moved_nib_img, moved_file)
    print(f'>>> {moved_file}')
    # break

In [ ]:
import time
from multiprocessing import Pool

subject = 'sub-07142303'
run = '10'

data_path = Path('/run/user/1000/gvfs/sftp:host=30.30.30.17,user=youjia/mnt/ngshare2/TRIGGER/BIDS')
output_path = Path('/run/user/1000/gvfs/sftp:host=30.30.30.17,user=youjia/mnt/ngshare2/TRIGGER/Preprocessing')
fixed_image = get_fixed_image(get_ants_image(data_path, subject, run, skip_frame=1000)[0])

for run in ['10', '11', '12', '13', '14']:
    moving, moving_file_path = get_ants_image(data_path, subject, run, skip_frame=1000)
    moved_file = output_path / subject / moving_file_path.name.replace('.nii.gz', '_mc.nii.gz')
    print(moving_file_path)
    print(moved_file)
    if not moved_file.parent.exists():
        moved_file.parent.mkdir(parents=True, exist_ok=True)
    if moved_file.exists():
        print(moved_file, ': exists')
        continue
    
    cmd_list = list() 
    for i in range(0, 10000):
        cmd_list.append([moving[..., i], fixed_image[:], i])
    time_s = time.time()
    pool = Pool(20)
    results = pool.starmap(motion_detection, cmd_list)
    del cmd_list
    pool.close()
    pool.join()
    print(time.time() - time_s)
    
    results = np.array(results, dtype=object)
    datas = {
    'frame': results[:, 0].astype(float),
    'mse': results[:, 1].astype(float),
    'x_r': results[:, 4].astype(float),
    'y_r': results[:, 3].astype(float),
    'x_t': results[:, 7].astype(float),
    'y_t': results[:, 6].astype(float),
    'x_s': results[:, 5].astype(float),
    'y_s': results[:, 2].astype(float),
    }
    datas['abs_dis'] = np.sqrt(np.power(datas['x_t'], 2), np.power(datas['y_t'], 2))
    datas['rel_dis'] = datas['abs_dis'].copy()
    datas['rel_dis'][1:] = np.abs(datas['abs_dis'][1:] - datas['abs_dis'][:-1])
    datas['rel_dis'][0] = 0
    motion_df = pd.DataFrame(datas, index=datas['frame'])
    motion_df[['x_t', 'y_t']].plot()
    plt.show()
    motion_df[['x_r', 'y_r']].plot()
    plt.show()
    motion_df[['mse']].plot()
    plt.show()
    motion_df[['abs_dis', 'rel_dis']].plot()
    plt.show()

    moved_mc_csv_file = output_path / subject / moving_file_path.name.replace('.nii.gz', '_mc.csv')
    motion_df.to_csv(moved_mc_csv_file, index=False)
    
    moving_nib_img = nib.load(str(moving_file_path))
    moved = np.array(list(results[:, 8]))
    moved = moved[..., np.newaxis]
    moved = np.transpose(moved, [1, 2, 3, 0])
    moved = np.concatenate((moving_nib_img.dataobj[..., :1000], moved), axis=3)
    
    moved_nib_img = nib.Nifti1Image(moved, moving_nib_img.affine, moving_nib_img.header)
    
    nib.save(moved_nib_img, moved_file)
    print(f'>>> {moved_file}')
    # break

In [ ]:
import time
from multiprocessing import Pool

subject = 'sub-07142304'
run = '06'

data_path = Path('/run/user/1000/gvfs/sftp:host=30.30.30.17,user=youjia/mnt/ngshare2/TRIGGER/BIDS')
output_path = Path('/run/user/1000/gvfs/sftp:host=30.30.30.17,user=youjia/mnt/ngshare2/TRIGGER/Preprocessing')
fixed_image = get_fixed_image(get_ants_image(data_path, subject, run, skip_frame=1000)[0])

for run in ['06', '07', '08', '09', '10']:
    moving, moving_file_path = get_ants_image(data_path, subject, run, skip_frame=1000)
    moved_file = output_path / subject / moving_file_path.name.replace('.nii.gz', '_mc.nii.gz')
    print(moving_file_path)
    print(moved_file)
    if not moved_file.parent.exists():
        moved_file.parent.mkdir(parents=True, exist_ok=True)
    if moved_file.exists():
        print(moved_file, ': exists')
        continue
    
    cmd_list = list() 
    for i in range(0, 10000):
        cmd_list.append([moving[..., i], fixed_image[:], i])
    time_s = time.time()
    pool = Pool(20)
    results = pool.starmap(motion_detection, cmd_list)
    del cmd_list
    pool.close()
    pool.join()
    print(time.time() - time_s)
    
    results = np.array(results, dtype=object)
    datas = {
    'frame': results[:, 0].astype(float),
    'mse': results[:, 1].astype(float),
    'x_r': results[:, 4].astype(float),
    'y_r': results[:, 3].astype(float),
    'x_t': results[:, 7].astype(float),
    'y_t': results[:, 6].astype(float),
    'x_s': results[:, 5].astype(float),
    'y_s': results[:, 2].astype(float),
    }
    datas['abs_dis'] = np.sqrt(np.power(datas['x_t'], 2), np.power(datas['y_t'], 2))
    datas['rel_dis'] = datas['abs_dis'].copy()
    datas['rel_dis'][1:] = np.abs(datas['abs_dis'][1:] - datas['abs_dis'][:-1])
    datas['rel_dis'][0] = 0
    motion_df = pd.DataFrame(datas, index=datas['frame'])
    motion_df[['x_t', 'y_t']].plot()
    plt.show()
    motion_df[['x_r', 'y_r']].plot()
    plt.show()
    motion_df[['mse']].plot()
    plt.show()
    motion_df[['abs_dis', 'rel_dis']].plot()
    plt.show()

    moved_mc_csv_file = output_path / subject / moving_file_path.name.replace('.nii.gz', '_mc.csv')
    motion_df.to_csv(moved_mc_csv_file, index=False)
    
    moving_nib_img = nib.load(str(moving_file_path))
    moved = np.array(list(results[:, 8]))
    moved = moved[..., np.newaxis]
    moved = np.transpose(moved, [1, 2, 3, 0])
    moved = np.concatenate((moving_nib_img.dataobj[..., :1000], moved), axis=3)
    
    moved_nib_img = nib.Nifti1Image(moved, moving_nib_img.affine, moving_nib_img.header)
    
    nib.save(moved_nib_img, moved_file)
    print(f'>>> {moved_file}')
    # break

In [ ]:
import time
from multiprocessing import Pool

subject = 'sub-07142304'
run = '11'

data_path = Path('/run/user/1000/gvfs/sftp:host=30.30.30.17,user=youjia/mnt/ngshare2/TRIGGER/BIDS')
output_path = Path('/run/user/1000/gvfs/sftp:host=30.30.30.17,user=youjia/mnt/ngshare2/TRIGGER/Preprocessing')
fixed_image = get_fixed_image(get_ants_image(data_path, subject, run, skip_frame=1000)[0])

for run in ['11', '12', '13', '14', '15']:
    moving, moving_file_path = get_ants_image(data_path, subject, run, skip_frame=1000)
    moved_file = output_path / subject / moving_file_path.name.replace('.nii.gz', '_mc.nii.gz')
    print(moving_file_path)
    print(moved_file)
    if not moved_file.parent.exists():
        moved_file.parent.mkdir(parents=True, exist_ok=True)
    if moved_file.exists():
        print(moved_file, ': exists')
        continue
    
    cmd_list = list() 
    for i in range(0, 10000):
        cmd_list.append([moving[..., i], fixed_image[:], i])
    time_s = time.time()
    pool = Pool(20)
    results = pool.starmap(motion_detection, cmd_list)
    del cmd_list
    pool.close()
    pool.join()
    print(time.time() - time_s)
    
    results = np.array(results, dtype=object)
    datas = {
    'frame': results[:, 0].astype(float),
    'mse': results[:, 1].astype(float),
    'x_r': results[:, 4].astype(float),
    'y_r': results[:, 3].astype(float),
    'x_t': results[:, 7].astype(float),
    'y_t': results[:, 6].astype(float),
    'x_s': results[:, 5].astype(float),
    'y_s': results[:, 2].astype(float),
    }
    datas['abs_dis'] = np.sqrt(np.power(datas['x_t'], 2), np.power(datas['y_t'], 2))
    datas['rel_dis'] = datas['abs_dis'].copy()
    datas['rel_dis'][1:] = np.abs(datas['abs_dis'][1:] - datas['abs_dis'][:-1])
    datas['rel_dis'][0] = 0
    motion_df = pd.DataFrame(datas, index=datas['frame'])
    motion_df[['x_t', 'y_t']].plot()
    plt.show()
    motion_df[['x_r', 'y_r']].plot()
    plt.show()
    motion_df[['mse']].plot()
    plt.show()
    motion_df[['abs_dis', 'rel_dis']].plot()
    plt.show()

    moved_mc_csv_file = output_path / subject / moving_file_path.name.replace('.nii.gz', '_mc.csv')
    motion_df.to_csv(moved_mc_csv_file, index=False)
    
    moving_nib_img = nib.load(str(moving_file_path))
    moved = np.array(list(results[:, 8]))
    moved = moved[..., np.newaxis]
    moved = np.transpose(moved, [1, 2, 3, 0])
    moved = np.concatenate((moving_nib_img.dataobj[..., :1000], moved), axis=3)
    
    moved_nib_img = nib.Nifti1Image(moved, moving_nib_img.affine, moving_nib_img.header)
    
    nib.save(moved_nib_img, moved_file)
    print(f'>>> {moved_file}')
    # break

# 转为mm展示结果

# 回归头动参数